# Analyzing Air Quality In China

This notebook explores syntehtic dataset containing air pollution data from 5 major cities in China: Beijing, Shanghai, Guangzhou, Chengdu, and Shenzhen.

The data spans from 2015 to 2025 and includes data for various air pollutants and weather data.

### Objectives:
- **Visualize** Air Quality Trends over time across cities
- **Compare** pollution and particulate matter levels between cities, time of day, season, and weather conditions.
- **Explore** data using dropdowns and animations by using `ipywidgets` and `plotly.express`
- **Forecast** AQI levels using regression model from `scikit-learn`
- **Evaluate** and compare different machine learning models for accuracy and reliability.

This analysis builds upon core concepts covered in OMAT5100M: Programming for Data Science, while also incorporating additional techniques and tools beyond the scope of the course to demonstrate independent learning and application.

**Dataset Source:**
Air Pollution in China (2015-2025)
Available on [Kaggle](https://www.kaggle.com/datasets/khushikyad001/air-pollution-in-china-2015-2025)
Accessed on: April 13, 2025.


## Importing Libraries

In [685]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from ipywidgets import widgets
from plotly.subplots import make_subplots

## Loading & Previewing Dataset

In [686]:
pollution = pd.read_csv("air_pollution_china.csv")
pollution.info() # getting a summary of the dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   PM2.5 (µg/m³)       3000 non-null   float64
 1   PM10 (µg/m³)        3000 non-null   float64
 2   NO2 (µg/m³)         3000 non-null   float64
 3   SO2 (µg/m³)         3000 non-null   float64
 4   CO (mg/m³)          3000 non-null   float64
 5   O3 (µg/m³)          3000 non-null   float64
 6   Temperature (°C)    3000 non-null   float64
 7   Humidity (%)        3000 non-null   float64
 8   Wind Speed (m/s)    3000 non-null   float64
 9   Wind Direction (°)  3000 non-null   float64
 10  Pressure (hPa)      3000 non-null   float64
 11  Precipitation (mm)  3000 non-null   float64
 12  Visibility (km)     3000 non-null   float64
 13  AQI                 3000 non-null   int64  
 14  Season              3000 non-null   object 
 15  City                3000 non-null   object 
 16  Latitu

Upon inspection, we can see that none of the values in the dataframe are null values. In this case, we can proceed with an exploratory data analysis.

In [687]:
pollution.describe() # Checking summary statistics

,PM2.5 (µg/m³),PM10 (µg/m³),NO2 (µg/m³),SO2 (µg/m³),CO (mg/m³),O3 (µg/m³),Temperature (°C),Humidity (%),Wind Speed (m/s),Wind Direction (°),Pressure (hPa),Precipitation (mm),Visibility (km),AQI,Latitude,Longitude,Hour,Month,Year,Station ID
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,130.072742,158.818398,52.948697,25.426127,2.562514,105.613365,15.117060,54.737773,5.137389,179.396411,1014.894063,24.880798,9.900773,249.639667,35.037328,110.072445,11.289667,6.537333,2019.555333,50.164667
std,68.140050,80.915207,26.905603,14.177923,1.409570,54.054354,14.719677,26.187670,2.891444,104.076767,20.348410,14.638862,5.751453,143.396118,8.666335,11.604091,6.835057,3.441204,2.850669,28.807838
min,10.039019,20.017134,5.005047,1.014693,0.102207,10.094551,-9.997818,10.049686,0.002052,0.017217,980.020253,0.002508,0.103173,0.000000,20.005034,90.006533,0.000000,1.000000,2015.000000,1.000000
25%,72.409621,86.982912,29.885955,12.720078,1.362659,59.561719,2.123508,31.915813,2.601402,87.632382,997.419156,12.036467,4.997298,126.000000,27.495695,100.001894,5.750000,4.000000,2017.000000,26.000000
50%,128.858172,157.174723,53.404879,25.677423,2.553778,106.706302,15.334326,53.993836,5.206242,178.820576,1014.296950,24.785057,9.850522,248.000000,35.148201,110.181877,11.000000,7.000000,2020.000000,49.000000
75%,188.579931,228.603910,75.955623,37.755735,3.785613,151.293020,28.016646,77.409183,7.750817,268.835379,1032.671602,37.617215,14.808608,374.000000,42.612931,119.977826,17.000000,10.000000,2022.000000,75.000000
max,249.847473,299.702293,99.979511,49.989850,4.998658,199.936387,39.959957,99.998108,9.991277,359.881532,1049.997553,49.984126,19.998597,499.000000,49.998873,129.989284,23.000000,12.000000,2024.000000,99.000000


## Pollution Trends Over Time

In [688]:
city_year_grouped_df = pd.DataFrame(pollution.groupby(["City", "Year"])[["PM2.5 (µg/m³)", 'AQI']].mean().reset_index()) # Grouping data by city and year
city_year_grouped_df = city_year_grouped_df.melt(id_vars=["City", "Year"], var_name='Pollutant', value_name='Concentration')

In [689]:
fig = px.line(city_year_grouped_df, x = 'Year', y = 'Concentration', facet_row='Pollutant', color='City', markers=True, title='PM2.5 Trends by City Over Time')
print('Tip: Hover over the datapoints to preview data labels')
fig.update_yaxes(matches=None)
fig.update_layout(height=800, width=1200)
fig.show()

Tip: Hover over the datapoints to preview data labels


There seems to be a lot of variability in the PM2.5 particulate matter concentration year on year. It would be good to understand variability, the median, and the extreme in PM2.5 readings for each city using a box plot.

## City Wise Pollution Comparision

In [690]:
fig = px.box(pollution, x = 'City', y = 'PM2.5 (µg/m³)', color='City', title='PM2.5 Trends by City Over Time')
fig.show()

The median PM2.5 matter value seems to be higher in Shanghai and the lowest in Beijing but only by a small margin. Overall, the cities seem to have close median and interquartile range.

It would also be good to check PM2.5 matter concentrations for different seasons and cities.

In [691]:
city_season_df = pollution[['City', 'Season', 'PM2.5 (µg/m³)']]
city_season_df.head()


,City,Season,PM2.5 (µg/m³)
0,Shenzhen,Spring,94.437337
1,Shanghai,Spring,194.174790
2,Beijing,Autumn,45.037661
3,Shanghai,Summer,76.131857
4,Beijing,Spring,204.127929


In [692]:
# Creating a multi select filter using ipywidgets
city_filter = widgets.SelectMultiple(
    options=city_season_df['City'].unique(),
    value=['Beijing'],
    description='City',
    layout=widgets.Layout(width='50%'))

In [693]:
def update_plot(input_cities):
    filtered_df = city_season_df[city_season_df['City'].isin(input_cities)]
    fig = px.box(
        filtered_df,
        x = 'Season',
        y = 'PM2.5 (µg/m³)',
        title = 'PM2.5 (µg/m³) concentration by season'
    )
    fig.update_traces(marker_color='#ff7f0e', line_color='#ff7f0e') # changing the color of the boxplot
    fig.update_xaxes(categoryorder='category ascending') # ordering x axis by season
    fig.show()

print("Tip: Hold Cmd (or Ctrl on Windows) and click to select multiple cities.")
widgets.interactive(update_plot, input_cities=city_filter)

Tip: Hold Cmd (or Ctrl on Windows) and click to select multiple cities.


interactive(children=(SelectMultiple(description='City', index=(2,), layout=Layout(width='50%'), options=('She…

On an aggregate across all cities combined, spring seems to have the highest median PM2.5 concentration but this trend doesn't seem to be the case for individual cities. For example, Shanghai has higher PM2.5 concentration during Autumn and Summer.

Another thing to note is that Chengdu seems to have the highest median PM2.5 concentration of 160 µg/m³.

## Pollution levels and weather

In this section, we'll explore patterns between weather conditions and different pollutants. We'll also explore correlation between different pollutants

In [694]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Listing all weather and pollutants for filters
weather_conditions = ['Temperature (°C)', 'Humidity (%)', 'Precipitation (mm)', 'Wind Speed (m/s)']
pollutants = ['PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NO2 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)', 'O3 (µg/m³)', 'AQI']

# Creating filter to select pollutant
pollutant_filter = widgets.Dropdown(
    options=pollutants,
    value='PM2.5 (µg/m³)',
    description='Pollutant:',
    disabled=False,
)

# Creating filter to select city
city_filter = widgets.Dropdown(
    options=pollution['City'].unique(),
    value='Beijing',
    description='City:',
    disabled=False,
)

# Creating filter to select weather
weather_filter = widgets.Dropdown(
    options=weather_conditions,
    value='Temperature (°C)',
    description='Weather:',
    disabled=False,
)

def update_weather_pollution_plot(input_pollutant, input_city, input_weather):

    # filtering data based on user selected input
    filtered_df = pollution[pollution['City'] == input_city][['City', input_pollutant, input_weather, 'Season', 'Year']].sort_values(by=['Year'])

    # Creating a scatter plot and faceting on Season
    fig = px.scatter(
        filtered_df,
        x = input_weather,
        y = input_pollutant,
        facet_row='Season',
        animation_frame='Year'
    )

    fig.update_layout(height=1000, width=800, title_text=f"{input_pollutant} vs {input_weather}s for {input_city}")
    fig.show()

print('Tip: Press the Play button at the bottom to view trends by year')
widgets.interactive(update_weather_pollution_plot, input_city=city_filter, input_pollutant=pollutant_filter, input_weather=weather_filter)

Tip: Press the Play button at the bottom to view trends by year


interactive(children=(Dropdown(description='Pollutant:', options=('PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NO2 (µg/m³…

In [695]:
# Checking correlation between different pollutants
pollutants_data = pollution[pollutants]

corr_matrix = pollutants_data.corr() # Creating correlation matrix

# Using heatmap to visualize the correlation matrix
fig = px.imshow(
    corr_matrix,
    text_auto=True,
    color_continuous_scale='balance',
    zmin=-1, zmax=1,
    title='Correlation Matrix Heatmap'
)

fig.show()

There seems to be no discrenable correlation different various pollutants.

## Pollutant Concentration by Different Timeframes

In this section we'll investigate pollutant concentration for different timeframes of the day to identify rush hour or late night pollutant patterns.

In [696]:
pollution_by_hour_month = pollution[['Hour', 'Month', 'PM2.5 (µg/m³)','AQI', 'PM10 (µg/m³)', 'NO2 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)', 'O3 (µg/m³)']].copy()
pollution_by_hour_month.rename(columns={'CO (mg/m³)': 'CO (µg/m³)'}, inplace=True)

pollution_by_hour_month.loc[:, 'CO (µg/m³)'] = pollution_by_hour_month['CO (µg/m³)']*1000

pollution_by_hour = pollution_by_hour_month.melt(id_vars=['Hour'], var_name='Pollutant', value_name='Concentration')

pollution_by_hour = pollution_by_hour.groupby(['Hour', 'Pollutant']).median().reset_index()

fig = px.line(
    pollution_by_hour,
    x = 'Hour',
    y = 'Concentration',
    facet_row='Pollutant',
    title='Hourly Pollutant Concentration'
)
fig.update_yaxes(matches=None)
fig.update_layout(height=1500, width=1200, title_text="Pollutant Concentration by Hour")
fig.show()

In [697]:
pollution_by_month = pollution_by_hour_month.melt(id_vars=['Month'], var_name='Pollutant', value_name='Concentration')
pollution_by_month = pollution_by_month.groupby(['Month', 'Pollutant']).median().reset_index()

fig = px.line(
    pollution_by_month,
    x = 'Month',
    y = 'Concentration',
    facet_row='Pollutant',
    title='Monthly Pollutant Concentration'
)
fig.update_yaxes(matches=None)
fig.update_layout(height=1500, width=1200, title_text="Pollutant Concentration by Month")
fig.show()

## Forecasting AQI Levels

We'll use regression models offered by sklearn to predict AQI value using numerical features only. 

Note: For the sake of simplicity, and due to the limited knowledge on handling categorical varibles in machine learning, I have excluded categorical varibles from features. This allowed me to focus on the numerical features and the regression techniques taught in the course. 

### Linear Regression

In [698]:
# Importing all required modules
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [699]:
features = ['PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NO2 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)', 'O3 (µg/m³)', 'Temperature (°C)', 'Humidity (%)', 'Precipitation (mm)', 'Wind Speed (m/s)', 'Pressure (hPa)', 'Precipitation (mm)', 'Visibility (km)']

y = pollution.loc[:, 'AQI'] # we'll use PM2.5 value as the variable to predict ie target
X = pollution.loc[:, features] # we'll use all the other variables as input variables ie features

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80)


We'll now scale the data.

This process ensures all the data is on the same scale. The equation is:

$z = \frac{x - \mu}{\sigma}$

In [700]:
# Scaling the data

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [701]:
linear_regressor = LinearRegression()

linear_regressor.fit(X_train_scaled, y_train)
y_pred = linear_regressor.predict(X_test_scaled)

# The coefficients
print("Coefficients: \n", linear_regressor.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

Coefficients: 
 [-0.71492543 -1.70795421  0.08335521 -5.08145412 -2.89871094  1.73265651
  5.25912711  3.57372943  0.82444235 -1.09098814 -2.39370689  0.82444235
  0.53051473]
Mean squared error: 19994.97
Coefficient of determination: -0.01


Given the coefficient of determination is 0, our model is no better than average when it comes to predicting the target.

In [702]:
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

fig = px.scatter(results_df, x='Actual', y='Predicted', title='Actual vs Predicted')

fig.add_shape(
    type='line',
    x0=results_df['Actual'].min(), y0=results_df['Actual'].min(),
    x1=results_df['Actual'].max(), y1=results_df['Actual'].max(),
    line=dict(color='red', dash='dash'),
    name='Ideal Fit'
)

fig.update_layout(
    xaxis_title='Actual',
    yaxis_title='Predicted',
    width=700,
    height=500,
    xaxis_range=[-50, 550],
    yaxis_range=[200,300]
)

fig.show()

### Other Alternatives

Given the linear regression model has a poor fit, we'll test other regression models to check if we can get better accuracy. Here are some of the models we'll try out:

- **Polynomial Regression** 
- **Random Forest Regressor** 
- **Ridge Regression** 
- **Lasso Regression** 

### Polynomial Regression

We'll apply polynomial regression as the behaviour between the target and features is non linear. Polynomials will take the form below (for a simple case of just one feature):

$$
y = \beta_0 + \beta_1 x + \beta_2 x^2
$$

In [703]:
from sklearn.preprocessing import PolynomialFeatures

In [704]:
def apply_polynomial_regression(degrees):
    poly_features = PolynomialFeatures(degree=degrees)
    X_train_poly = poly_features.fit_transform(X_train)
    X_test_poly = poly_features.transform(X_test)

    # We'll now scale these
    poly_scaler = StandardScaler()
    X_train_poly_scaled = poly_scaler.fit_transform(X_train_poly)
    X_test_poly_scaled = poly_scaler.transform(X_test_poly)

    model = LinearRegression()

    model.fit(X_train_poly_scaled, y_train)

    y_pred_poly = model.predict(X_test_poly_scaled)

    mse = mean_squared_error(y_test, y_pred_poly)
    r2 = r2_score(y_test, y_pred_poly)

    print(f"Polynomial Regression (degree={degrees}) — MSE: {mse:.2f}, R²: {r2:.2f}")
    
    return r2

In [705]:
# We'll try different degrees of polynomial regression to see which one performs the best. Anything more than 5 causes significant time to compute so we'll go as high as 5. 
r2_polynomial = []
for i in range(1,6):
    r2_polynomial.append(apply_polynomial_regression(i))

px.line(x=list(range(1,6)), y=r2_polynomial)

Polynomial Regression (degree=1) — MSE: 19994.97, R²: -0.01
Polynomial Regression (degree=2) — MSE: 20608.04, R²: -0.04
Polynomial Regression (degree=3) — MSE: 25716.98, R²: -0.30
Polynomial Regression (degree=4) — MSE: 123948.27, R²: -5.26
Polynomial Regression (degree=5) — MSE: 228668.96, R²: -10.55


Generally, the performance seems to get poorer as we increase degrees (especially after 3rd degree) so polynomial regression may not be the best approach based on computing power required and model prediction up to 5th degree polynomial.

### Random Forest Regression

A random forest is a meta estimator that fits a number of decision tree regressors on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting (scikit-learn, 2024).

**Reference:**

Scikit-learn developers. (2024). *Random forest regression — scikit-learn 1.6.1 documentation*. Retrieved from https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html


In [706]:
from sklearn.ensemble import RandomForestRegressor

In [707]:

random_forest_model = RandomForestRegressor() # We'll use the model with default arguements

random_forest_model.fit(X_train_scaled, y_train)

y_pred_random_forest = random_forest_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred_random_forest)
r2 = r2_score(y_test, y_pred_random_forest)

print(f"Random Forest Regression — MSE: {mse:.2f}, R²: {r2:.2f}")
    

Random Forest Regression — MSE: 20511.31, R²: -0.04


### Ridge Regression

In [708]:
rdg = Ridge()

rdg.fit(X_train_scaled, y_train)

y_pred_ridge = rdg.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred_ridge)
r2 = r2_score(y_test,y_pred_ridge)

print(f"Ridge Regression — MSE: {mse:.2f}, R²: {r2:.2f}")

Ridge Regression — MSE: 19994.84, R²: -0.01


In [709]:
lasso = Lasso()

lasso.fit(X_train_scaled, y_train)

y_pred_ridge = lasso.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred_ridge)
r2 = r2_score(y_test,y_pred_ridge)

print(f"Ridge Regression — MSE: {mse:.2f}, R²: {r2:.2f}")

Ridge Regression — MSE: 19920.04, R²: -0.01


Overall, none of the models tested led to accurate predictions. This could be due to the following factors:

- **Synthetic Data** Since the dataset is synthetic, model performance heavily depends on how the data was generated. If it was based on real-world data with added noise, the models would likely capture underlying patterns more effectively. However, if the data was generated from scratch without any meaningful structure, it's possible that no real correlations exist for the models to learn from. This aligns with the correlation heatmap where all variables showed little to no correlation with each other except themselves.

- **Disregarding Categorical Variables** Categorical variables such as City, Season, or Weather Condition were excluded from the models for simplicity. If these variables held predictive power, their exclusion may have significantly limited the model’s ability to identify patterns and make accurate predictions.

- **Target Variable Selection** We could test the models with a different target varible such as PM2.5.

A summary of all models can be found below:

In [710]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regression': RandomForestRegressor(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
}

final_results = []

for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test,y_pred)
    
    final_results.append([model_name, mse, r2])
    
final_results = pd.DataFrame(final_results, columns=['Model', 'Mean Squared Error', 'R2'])
final_results

,Model,Mean Squared Error,R2
0,Linear Regression,19994.966611,-0.010137
1,Random Forest Regression,20998.500795,-0.060835
2,Ridge Regression,19994.838252,-0.010131
3,Lasso Regression,19920.044033,-0.006352
